Ufuk Altun

using Pkg; pkg"add CUDA Knet IterTools FileIO JLD2 Images"

In [1]:
include("models.jl")
include("preprocess.jl")

PACS

In [2]:
batchsize = 128
atype = Array{Float32}   #Array{Float32}   CuArray{Float32}
data_path = "/Users/ufukaltun/Documents/koç/dersler/ku deep learning/project";
target = "cartoon"

painting, cartoon, photo, sketch, baseline = PACS(data_path, batchsize, atype, target);

In [3]:
baseline

19-element Data{Tuple{Array{Float32, N} where N, Array{Int32, N} where N}}

In [4]:
baseline_model =  Chain2( Conv2(11, 11, 3, 64; padding=2, stride=4),
                         Conv2( 5,  5, 64, 192; padding=2),
                         Conv1( 3,  3,  192, 384; padding=1),
                         Conv1( 3,  3,  384, 256; padding=1),
                         Conv2( 3,  3,  256, 256; padding=1),
                         Dense(256*6*6, 4096,pdrop=0.5),
                         Dense(4096, 4096, pdrop=0.5),
                         Dense(4096, 7, identity) ; λ1=5e-5);

In [5]:
GS_model =  Chain2( Conv2(11, 11, 3, 64; padding=2, stride=4),
                         Conv2( 5,  5, 64, 192; padding=2),
                         Conv1( 3,  3,  192, 384; padding=1),
                         Conv1( 3,  3,  384, 256; padding=1),
                         Conv2( 3,  3,  256, 256; padding=1),
                         Dense(256*6*6, 4096,pdrop=0.5),
                         Dense(4096, 4096, pdrop=0.5),
                         Dense(4096, 7, identity) ; λ1=5e-5);

In [6]:
num_iter = 1000;
regterm = 5e-5;
lrterm = 1e-5;

In [7]:
function trainresults_baseline(target_dom)
r = ((baseline_model(baseline), baseline_model(target_dom), accuracy(baseline_model,data=baseline), accuracy(baseline_model,data=target_dom))
             for x in takenth(progress(adam(baseline_model,take(cycle(baseline),num_iter))),20));
r = reshape(collect(Float32,flatten(r)),(4,:))
save("results_baseline.jld2","results",r)
save("weights_baseline.jld2","weights",baseline_model)
GC.gc(true) # To save gpu memory
end

trainresults_baseline (generic function with 1 method)

In [ ]:
trainresults_baseline(cartoon)

┣▏                   ┫ [0.90%, 9/1000, 05:22/09:56:30, 31.83s/i] 